## Attention 신경망 구현 및 학습

* konlp 설치 관련해서 OS관련해서 추가 설치들이 필요하니, 관련 내용 잘 확인 필요!!

<img src="img/img_41.png">

* tag 여러 종류  
  - JHannanum is a morphological analyzer and POS tagger written in Java, and developed by the Semantic Web Research Center (SWRC) at KAIST since 1999   
  - Kkma is a morphological analyzer and natural language processing system written in Java, developed by the Intelligent Data Systems (IDS) Laboratory at SNU.    
  - KOMORAN is a relatively new open source Korean morphological analyzer written in Java, developed by Shineware, since 2013.    
  - MeCab, originally a Japanese morphological analyzer and POS tagger developed by the Graduate School of Informatics in Kyoto University, was modified to MeCab-ko by the Eunjeon Project to adapt to the Korean language.    
  - Open Korean Text is an open source Korean tokenizer written in Scala, developed by Will Hohyon Ryu.

In [1]:
import random
import tensorflow as tf
from konlpy.tag import Okt
import jpype
from konlpy.tag import Kkma

In [2]:
# 혹시 konlp가 제대로 설치 되었는지 확인을 위해서...
# 위의 konlpy만 해서는 제대로 설치 되었는지 확인이 안 될 수 있음!!!!
okt = Okt()

## 하이퍼 파라미터

In [3]:
EPOCHS = 200
NUM_WORDS = 2000

## Encoder

In [4]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        # 입력이 one-hot-encdoing 형식으로 들어오면 embedding을 수행을 먼저
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # 그리고 LSTM에서 hidden state을 출력으로 던져주어야 이것을 활용해서 return을 활용할 수 있다...
        # return_sequence=True : 디코더에서 나오는 것 하나하나 알아야 하기에...
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    # 인코더의 경우에는 입력이 들어오면 이것을 바탕으로 히든, 셀 스테이트를 하도록 ...
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        # h,c는 test를 할 경우에는 다음으로 넘겨 주어야 forward  로 계산을 하면서 나갈 수 있으니..
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder

In [5]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        
        #****Attention을 추가함!!!! 
        self.att = tf.keras.layers.Attention()
        
        # 최종 단어 단에서 어떤 것을 할 것인지..선택..
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        # ***처음 들어오는 인코더 단에서 넘어오는 것들을 받고,,
        x, s0, c0, H = inputs
        x = self.emb(x)
        # ****S는 hidden state를 모두 모아둔 부분..--> 쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        # ***쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..(1차원을 3차원으로 확장) & 마지막 히든은 제외...해서 :-1까지..
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        # ****키와 value를 계산하고,,,
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

* 참고 사항   
 - super : 자식클래스 내에서 코드에서도 부모클래스를 호출할 수 있습니다.

In [6]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        # 1) 가장 기본적으로 필요한 Encoder, Deoder, sos, eos에 관련된 부분을 지정을 함!!!!
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
    
    # 연결에서 구성 --> 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
    # 그리고 디코더에 입력을 넣어주어야 하기에..
    def call(self, inputs, training=False, mask=None):
       # (학습 과정)
        if training is True:
            # 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
            # 그리고 디코더에 입력을 넣어주어야 하기에..
            x, y = inputs
            # encoder에 입력  x를 넣어서 나온 hidden state, cell  (LSTM으로 구현이 되어서..)
            H, h, c = self.enc(x)
            # 최종 출력 y를 하도록...
            y, _, _ = self.dec((y, h, c, H))
            return y
        # (Test 과정) --> 그러니 정답  y는 없음..
        else:
            x = inputs
            H, h, c = self.enc(x)
            # 뒤에 디커더 부분에 입력을 넣어 주는 부분이 달라지게 된다!!!! --> sos  를 넣어준다.
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            # 최대 seq는 64길이까지만..
            seq = tf.TensorArray(tf.int32, 64)
            
            # tf의  for loop으로 사용을 해서 최대 64까지 
            for idx in tf.range(64):
                # for 에서 처음에는 처음으로 만들어준 sos를 decoder에 입력으로 넣어주고 --> self.dec(y,h,c,H)을 해서
                # 출력인 y와 hidden state, cell state를 출력으로 준다.
                y, h, c = self.dec([y, h, c, H])
                # 가장 큰 y값에 대한 index를 얻어오게 되는 과정
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                # reshape를 하면서 batch를 사용을 하기 위해서...
                y = tf.reshape(y, (1, 1))
                # seq를 하나씩 받으면서 처리하기 위해서...
                seq = seq.write(idx, y)
                # eos일 때 까지 for lopp순환
                if y == self.eos:
                    break
            # 
            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

<img src="img/img_40.png">

In [7]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    # 마지맞 eos는 있고, 처음 sos 는 없는 것...
    output_labels = labels[:, 1:]
    # 처음 sos는 포함이 되고, 마지막 eos는 빼고...
    shifted_labels = labels[:, :-1]
    # 그래서 위의 시프트 된 것들이 아래의 model의 학습 과정에 들어가게 된다!!!!!
    with tf.GradientTape() as tape:
        # 위의 output_labels, shifted_labels 이 들어가게 된다...
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [8]:
# Q-A 형태의 데이터 셋... ---> 본인들이 구성을 하시면 됩니다!!!!
# cafe.xlsx를 바탕으로 한 것이고, 다른 데이터 셋도 구할 수 있습니다!!!!!
dataset_file = 'chatbot_data_sample_2.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r',encoding='utf-8') as file:
    lines = file.readlines()
    # 한 줄에 대한 형태소 분석 수행-->morphs
    seq = [' '.join(okt.morphs(line)) for line in lines]

# 원본 데이터에서 질문과 답변에 대한 것들을 분리...한줄 한줄 건너서 있으니...처음에 질문부터이니 0부터 질문, 홀수가 답변--> 하나씩 건너서..
questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

# 질문을 보고 한 번 데이터 셋을 섞어준다..
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)
# 섞으면서 train/ test로 구분을 하기 위한 것..
train_q = list()
train_a = list()
test_q = list()
test_a = list()

# 질문과 답변에 대한 것을 돌면서...
for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    # 5/1은 test, 5/4는 train
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

# 문장에서 짤라주는 것에 대한 세팅...
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

# 입력은 뒤로...패딩..
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
# 출력은 앞에로 패딩..
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [9]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [10]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 6.0906476974487305, Accuracy: 72.18946838378906
Epoch 2, Loss: 0.8242252469062805, Accuracy: 90.49724578857422
Epoch 3, Loss: 0.6410475373268127, Accuracy: 90.59552001953125
Epoch 4, Loss: 0.5597950220108032, Accuracy: 91.11634826660156
Epoch 5, Loss: 0.5483094453811646, Accuracy: 91.29324340820312
Epoch 6, Loss: 0.5363156199455261, Accuracy: 90.6446533203125
Epoch 7, Loss: 0.5181447863578796, Accuracy: 91.16548919677734
Epoch 8, Loss: 0.5107572078704834, Accuracy: 91.22444915771484
Epoch 9, Loss: 0.5061613917350769, Accuracy: 91.41116333007812
Epoch 10, Loss: 0.5029914975166321, Accuracy: 91.41116333007812
Epoch 11, Loss: 0.49557772278785706, Accuracy: 91.17530822753906
Epoch 12, Loss: 0.49373659491539, Accuracy: 91.1949691772461
Epoch 13, Loss: 0.49131107330322266, Accuracy: 91.41116333007812
Epoch 14, Loss: 0.48679834604263306, Accuracy: 91.14582824707031
Epoch 15, Loss: 0.4828869700431824, Accuracy: 91.30307006835938
Epoch 16, Loss: 0.4797080159187317, Accuracy: 91.3

Epoch 129, Loss: 0.10788575559854507, Accuracy: 97.97563171386719
Epoch 130, Loss: 0.10573737323284149, Accuracy: 98.04441833496094
Epoch 131, Loss: 0.10231230407953262, Accuracy: 98.11321258544922
Epoch 132, Loss: 0.09878571331501007, Accuracy: 98.13285827636719
Epoch 133, Loss: 0.09604290872812271, Accuracy: 98.19181823730469
Epoch 134, Loss: 0.0934770479798317, Accuracy: 98.24095916748047
Epoch 135, Loss: 0.0909980908036232, Accuracy: 98.3392333984375
Epoch 136, Loss: 0.08784918487071991, Accuracy: 98.35887908935547
Epoch 137, Loss: 0.08556298911571503, Accuracy: 98.3785400390625
Epoch 138, Loss: 0.08323968946933746, Accuracy: 98.3785400390625
Epoch 139, Loss: 0.08114688843488693, Accuracy: 98.4571533203125
Epoch 140, Loss: 0.0787009745836258, Accuracy: 98.4964599609375
Epoch 141, Loss: 0.07576128840446472, Accuracy: 98.52593994140625
Epoch 142, Loss: 0.07437480986118317, Accuracy: 98.63404083251953
Epoch 143, Loss: 0.07074864953756332, Accuracy: 98.73230743408203
Epoch 144, Loss: 0

## 테스트 루프

In [11]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('Question : ', test_text)
    print('Real Answer : ', gt_text)
    print('Prediction Answer: ', texts)

_
Question :  ['저 다 에 되나요 \n']
Real Answer :  ['\t 네 에 됩니다 \n']
Prediction Answer:  ['와이파이 암호 는 가나다라 입니다 \n']
_
Question :  ['아이스 카푸치노 컵 에 주실 수 있나요 \n']
Real Answer :  ['\t 컵 사이즈 가 음료 가 다 안 \n']
Prediction Answer:  ['네 가능합니다 \n']
_
Question :  ['네 이 할인 사용 할게요 \n']
Real Answer :  ['\t 네 드시고 가시나요 \n']
Prediction Answer:  ['네 500원 할인 되세요 \n']
_
Question :  ['와이파이 비밀번호 는 뭐 에요 \n']
Real Answer :  ['\t 에 \n']
Prediction Answer:  ['와이파이 비밀번호 는 종이 에 써져있습니다 \n']
_
Question :  ['밀크 티 있나요 \n']
Real Answer :  ['\t 네 있습니다 \n']
Prediction Answer:  ['네 캐리어 에 2 나왔습니다 \n']
_
Question :  ['밀크 티 에 혹시 우유 가 우유 이 \n']
Real Answer :  ['\t 저희 는 우유 을 사용 하고 있어요 \n']
Prediction Answer:  ['유리잔 영업 으로 드릴가요 \n']
_
Question :  ['쿠폰 주시나요 \n']
Real Answer :  ['\t 네 드릴게요 만 요 \n']
Prediction Answer:  ['네 카드 받았습니다 \n']
_
Question :  ['아이스 아메리카노 한잔 이 요 \n']
Real Answer :  ['\t 드시고 \n']
Prediction Answer:  ['네 아이스 아메리카노 로 드릴 까요 \n']
_
Question :  ['번 에 로 하고 찍어주세요 \n']
Real Answer :  ['\t 네 \n']
Prediction Answer:  ['네 캐리어 에